# Generate Standard 205 YAML for Unitary Air-Conditioning

In [57]:
import yaml
import uuid
import csv

In [72]:
import re
from cStringIO import StringIO

def _fix_dump(dump, indentSize=2):
    stream = StringIO(dump)
    out = StringIO()
    pat = re.compile('(\s*)([^:]*)(:*)')
    last = None

    prefix = 0
    for s in stream:    
        indent, key, colon = pat.match(s).groups()
        if indent=="" and key[0]!= '-':
            prefix = 0 
        if last:
            if len(last[0])==len(indent) and last[2]==':':
                if all([
                        not last[1].startswith('-'), 
                        s.strip().startswith('-')
                        ]):
                    prefix += indentSize
            elif len(last[0])==len(indent):
                if all([
                        last[1].startswith('-'), 
                        not s.strip().startswith('-')
                        ]):
                    prefix -= indentSize
        out.write(" "*prefix+s)
        last = indent, key, colon
    return out.getvalue()

In [58]:
example = {}  # Top level dictionary

## ASHRAE 205 Header

In [59]:
a205 = {}
a205["rsid"] = "0002"
a205["rsver"] = 1
a205["id"] = str(uuid.uuid4())
a205["versionId"] = str(uuid.uuid4())
a205["name"] = "Example Unitary Air-Conditioner"
a205["dataDate"] = "2016-02-12"
a205["dataSource"] = "Synthetic (Generated by N. Kruis)"
a205["notes"] = "This dataset is purley for illustrative purposes. Use at your own risk."

example["ASHRAE205"] = a205

## General Data

In [60]:
gd = {}
gd["compressorControlMethod"] = "Staged"
gd["blowerId"] = "399b1308-09e6-4891-8cd7-ea40c86f7e40"
gd["blowerPosition"] = "Draw Through"
gd["nominalVolumetricAirFlowRate"] = 0.646
gd["refrigerantType"] = "R-140a"
gd["compressorType"] = "Scroll"
gd["expansionDeviceType"] = "TXV"

example["GeneralData"] = gd

## AHRI Residential Ratings

In [61]:
arr = {}
arr["certifiedReferenceNumber"] = 1234567
arr["testStandardVersion"] = "AHRI 210/240 2008"
arr["manufacturerSoftwareVersion"] = "2.1"
arr["outdoorUnitManufacturer"] = "Manufacturer"
arr["outdoorUnitModelNumber"] = "OU-1"
arr["indoorUnitManufacturer"] = "Manufacturer"
arr["indoorUnitModelNumber"] = "IU-1"
arr["seer"] = 13.0
arr["eer"] = 11.0
arr["capacity"] = 5.16e3
arr["cyclingDegradationCoefficient"] = 0.25
arr["blowerEfficiency"] = 226.

example["AHRIResidentialRatings"] = arr

## Performance Map (Cooling)

In [62]:
pmc = {}
ppc = {}
todb = []
tedb = []
tewb = []
v = []
p = []
c = []
qt = []
pd = []
qs = []

with open('map.csv','r') as mapfile:
    csvreader = csv.reader(mapfile)
    for row in csvreader:
        todb.append(float(row[0]))
        tewb.append(float(row[1]))
        tedb.append(float(row[2]))
        v.append(float(row[3]))
        c.append(1.0)
        p.append(float(row[4]))
        qt.append(float(row[5]))
        pd.append(float(row[6]))
        qs.append(float(row[7]))

ppc["outdoorCoilEnteringDryBulbTemperature"] = todb
ppc["indoorCoilEnteringWetBulbTemperature"] = tewb
ppc["indoorCoilEnteringDryBulbTemperature"] = tedb
ppc["indoorCoilVolumetricFlowRatio"] = v
ppc["compressorSpeedRatio"] = c
ppc["ambientAbsolutePressure"] = p
ppc["grossTotalCoolingCapacity"] = qt
ppc["grossSensibleCoolingCapacity"] = qs
ppc["grossCoolingPower"] = pd

pmc["PerformancePointsCooling"] = ppc

example["PerformanceMapCooling"] = pmc

## Performance Map Standby

In [63]:
pms = {}
pps = {}
pps["outdoorCoilEnvironmentDryBulbTemperature"] = [0.0, 5.0]
pps["grossPower"] = [50.0, 0.0]
pms["PerformancePointsStandby"] = pps

example["PerformanceMapStandby"] = pms

In [73]:
with open('unitary.yaml', 'w') as outfile:
    outfile.write( _fix_dump(yaml.dump(example, default_flow_style=False)) )